# **Quinto conjunto de tareas a realizar**

## Paquetes necesarios e inicializaciones

La siguiente práctica consta de dos partes principales, la primera es entrenar una red neuronal para diferenciar características faciales y tras esto, crear un filtro que use las soluciones de la red, y la segunda, consiste en la realización de un filtro de ámbito libre.

Para la realización de las tareas será necesario realizar las siguientes instalaciones y creación de un nuevo enviroment

Se van a realizar las importaciones necesarias para la ejecución de los consiguientes fragmentos de código

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image # Usaremos Pillow en lugar de tf.keras.utils
import cv2 # Usaremos OpenCV para guardar
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time
import copy
# from retinaface import RetinaFace
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau # Para el scheduler
from tqdm import tqdm # Para las barras de progreso
import matplotlib.pyplot as plt

En el siguiente fragmento se procede a crear una carpeta con las imágenes divididas en "test", "train" y "validation" para el posterior entrenemiento.

In [9]:
# --- 1. Configuración de Preparación ---
IMG_SIZE = 224
# Ruta a la carpeta original en tu escritorio
ORIGINAL_DATA_PATH = "C:/Users/ivanp/Desktop/UTKFaces/" 
# Carpeta donde se guardarán los datos divididos
ORGANIZED_DATA_DIR = "organized_data" 
CLASS_NAMES = ["joven", "medio", "anciano"]

# --- 2. Funciones de Carga y Guardado (versión PyTorch/Pillow) ---

def get_label_from_age(age):
    if 0 <= age <= 29: return 0  # Joven
    elif 30 <= age <= 59: return 1  # Medio
    elif age >= 60: return 2  # Anciano
    return None

def load_and_preprocess_data(path, img_size):
    images = []
    labels = []
    print("Iniciando carga de datos desde la carpeta original...")
    for filename in os.listdir(path):
        if not filename.endswith((".jpg", ".png", ".jpeg")):
            continue
        try:
            age_str = filename.split('_')[0]
            age = int(age_str)
            label = get_label_from_age(age)
            if label is not None:
                full_path = os.path.join(path, filename)
                
                # Usamos Pillow (PIL) para cargar y redimensionar
                img = Image.open(full_path).convert('RGB')
                img = img.resize((img_size, img_size), Image.Resampling.LANCZOS)
                
                # Convertimos a array numpy y normalizamos
                img_array = np.array(img) / 255.0
                
                images.append(img_array) 
                labels.append(label)
        except Exception as e:
            # print(f"Error cargando {filename}: {e}")
            pass
            
    print(f"Carga finalizada. Total de imágenes: {len(images)}")
    # Convertimos a float32 para PyTorch
    X = np.array(images, dtype=np.float32) 
    y = np.array(labels)
    return X, y

def save_split_data_to_folders(X_data, y_data, split_name, base_dir):
    print(f"\nGuardando imágenes del conjunto: {split_name}...")
    for i, (img, label) in enumerate(zip(X_data, y_data)):
        class_name = CLASS_NAMES[label]
        target_dir = os.path.join(base_dir, split_name, class_name)
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
            
        filename = f"img_{i}.png"
        filepath = os.path.join(target_dir, filename)
        
        # Usamos OpenCV (cv2) para guardar (revirtiendo normalización)
        # cv2 espera BGR, así que convertimos RGB -> BGR
        img_bgr = cv2.cvtColor((img * 255.0).astype(np.uint8), cv2.COLOR_RGB2BGR)
        cv2.imwrite(filepath, img_bgr)
        
    print(f"Imágenes de {split_name} guardadas en {base_dir}/{split_name}")

# --- 3. Ejecución Principal (Solo si se ejecuta este script) ---
# (Este 'if' es para que puedas importar las funciones en otros scripts si lo necesitas)
if __name__ == "__main__":
    if os.path.exists(ORGANIZED_DATA_DIR):
        print(f"La carpeta '{ORGANIZED_DATA_DIR}' ya existe. No se necesita preparación.")
    else:
        print(f"Carpeta '{ORGANIZED_DATA_DIR}' no encontrada. Iniciando organización...")
        
        X, y = load_and_preprocess_data(ORIGINAL_DATA_PATH, IMG_SIZE)
        
        print("Dividiendo datos...")
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
        print("División completa.")
        
        save_split_data_to_folders(X_train, y_train, "train", base_dir=ORGANIZED_DATA_DIR)
        save_split_data_to_folders(X_val, y_val, "validation", base_dir=ORGANIZED_DATA_DIR)
        save_split_data_to_folders(X_test, y_test, "test", base_dir=ORGANIZED_DATA_DIR)
        
        print("\n--- ¡Organización de carpetas completa! ---")

Carpeta 'organized_data' no encontrada. Iniciando organización...
Iniciando carga de datos desde la carpeta original...


KeyboardInterrupt: 

In [ ]:
# --- 0. Verificación de GPU (PyTorch) ---
print("--- Verificando la disponibilidad de la GPU (PyTorch) ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
else:
    print("---------------------------------------------------------")
    print(">>> ⚠️ ¡ADVERTENCIA! No se encontró ninguna GPU. <<<")
    print(">>> El entrenamiento se ejecutará en la CPU (será lento).")
    print("---------------------------------------------------------")
print(f"Usando dispositivo: {device}")
print("-" * 50)


# --- 1. Configuración Inicial ---
IMG_SIZE = 224
ORGANIZED_DATA_DIR = "C:/Users/ivanp/Desktop/organized_data/"   # Cambiar según sea necesario
BATCH_SIZE = 32

# --- Configuración del Entrenamiento en 2 Fases ---
EPOCHS_HEAD = 15
EPOCHS_TUNE = 50 # Épocas SOLO para el ajuste fino (total = HEAD + TUNE)
LR_HEAD = 0.001
LR_TUNE = 1e-5
EARLY_STOP_PATIENCE = 15 # Paciencia para la parada temprana

# Archivos de salida
csv_log_file = './train_results/training_log.csv'
best_model_file = './train_results/best_age_model.pth'
summary_csv_file = './train_results/training_history_summary.csv'

# --- 3. Cargar Datos (Estilo PyTorch) ---
print(f"Cargando datos desde la carpeta '{ORGANIZED_DATA_DIR}'...")

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(ORGANIZED_DATA_DIR, 'train'), data_transforms['train']),
    'val': datasets.ImageFolder(os.path.join(ORGANIZED_DATA_DIR, 'validation'), data_transforms['val']),
    'test': datasets.ImageFolder(os.path.join(ORGANIZED_DATA_DIR, 'test'), data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
print(f"Clases encontradas: {class_names}")
print(f"Imágenes de entrenamiento: {dataset_sizes['train']}, Validación: {dataset_sizes['val']}")
print("Carga de datos finalizada.")


# --- 4. Definir el Modelo (ResNet50 en PyTorch) ---
model = models.resnet50(weights='IMAGENET1K_V1')

# Congelar la base (FASE 1)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, len(class_names))
)

model = model.to(device)
criterion = nn.CrossEntropyLoss()

# --- 5. Lógica de Entrenamiento (2 Fases separadas) ---

# Preparar CSV logger
try:
    with open(csv_log_file, 'w') as f:
        f.write('epoch,train_loss,train_acc,val_loss,val_acc\n')
    print(f"\nLogs de cada época se guardarán en: {csv_log_file}")
except IOError as e:
    print(f"Error al escribir CSV: {e}")

# Dataframe para el historial completo
full_history_df = pd.DataFrame()
best_val_acc = 0.0
print(f"El mejor modelo se guardará en: {best_model_file}")

# --- Fase 1: Entrenamiento de la Capa Final (Head) ---

optimizer = optim.AdamW(model.fc.parameters(), lr=LR_HEAD, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

print(f"\n--- Iniciando Fase 1: Entrenamiento de la Capa Final ({EPOCHS_HEAD} épocas) ---")

for epoch in range(EPOCHS_HEAD):
    model.train()
    train_loss, correct_train, total_train = 0.0, 0, 0
    train_loop = tqdm(dataloaders['train'], desc=f"Epoch {epoch+1}/{EPOCHS_HEAD} [Head Train]", leave=False)
    
    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        train_loop.set_postfix(loss=loss.item())

    # Validación de la Fase 1
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_train_loss = train_loss / len(dataloaders['train'])
    avg_train_acc = (correct_train / total_train)
    avg_val_loss = val_loss / len(dataloaders['val'])
    avg_val_acc = (correct_val / total_val)
    
    # Guardar en CSV y DF
    try:
        with open(csv_log_file, 'a') as f:
            f.write(f'{epoch},{avg_train_loss:.4f},{avg_train_acc:.4f},{avg_val_loss:.4f},{avg_val_acc:.4f}\n')
        epoch_data = {'epoch': epoch, 'train_loss': avg_train_loss, 'train_acc': avg_train_acc, 'val_loss': avg_val_loss, 'val_acc': avg_val_acc}
        full_history_df = pd.concat([full_history_df, pd.DataFrame([epoch_data])], ignore_index=True)
    except Exception as e:
        print(f"Error guardando logs: {e}")
    
    # Imprimir
    print(f"Epoch {epoch+1}/{EPOCHS_HEAD} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc*100:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc*100:.2f}%")
    
    # Guardar Mejor Modelo
    if avg_val_acc > best_val_acc:
        print(f"  ¡Mejora! Guardando modelo... (Val Acc: {best_val_acc*100:.2f}% -> {avg_val_acc*100:.2f}%)")
        torch.save(model.state_dict(), best_model_file)
        best_val_acc = avg_val_acc
    
    scheduler.step(avg_val_acc)

# --- Fase 2: Ajuste Fino (Fine-Tuning) ---

print(f"\n--- Iniciando Fase 2: Ajuste Fino de todo el modelo ({EPOCHS_TUNE} épocas) ---")

# Descongelar TODAS las capas
for param in model.parameters():
    param.requires_grad = True

# Nuevo optimizador para TODOS los parámetros, con LR muy baja
optimizer = optim.AdamW(model.parameters(), lr=LR_TUNE, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)

# Reiniciar variables de Early Stopping
epochs_no_improve = 0
min_delta = 0.0001 # 0.01%

for epoch in range(EPOCHS_TUNE):
    current_total_epoch = epoch + EPOCHS_HEAD # Época total
    
    model.train()
    train_loss, correct_train, total_train = 0.0, 0, 0
    train_loop = tqdm(dataloaders['train'], desc=f"Epoch {current_total_epoch+1}/{EPOCHS_HEAD + EPOCHS_TUNE} [Tune Train]", leave=False)
    
    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        train_loop.set_postfix(loss=loss.item())

    # Validación de la Fase 2
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    avg_train_loss = train_loss / len(dataloaders['train'])
    avg_train_acc = (correct_train / total_train)
    avg_val_loss = val_loss / len(dataloaders['val'])
    avg_val_acc = (correct_val / total_val)
    
    # Guardar en CSV y DF
    try:
        with open(csv_log_file, 'a') as f:
            f.write(f'{current_total_epoch},{avg_train_loss:.4f},{avg_train_acc:.4f},{avg_val_loss:.4f},{avg_val_acc:.4f}\n')
        epoch_data = {'epoch': current_total_epoch, 'train_loss': avg_train_loss, 'train_acc': avg_train_acc, 'val_loss': avg_val_loss, 'val_acc': avg_val_acc}
        full_history_df = pd.concat([full_history_df, pd.DataFrame([epoch_data])], ignore_index=True)
    except Exception as e:
        print(f"Error guardando logs: {e}")
    
    # Imprimir
    print(f"Epoch {current_total_epoch+1}/{EPOCHS_HEAD + EPOCHS_TUNE} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc*100:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc*100:.2f}%")

    # Lógica de Scheduler y Early Stopping
    scheduler.step(avg_val_acc)
    
    if avg_val_acc > (best_val_acc + min_delta):
        print(f"  ¡Mejora! Guardando modelo... (Val Acc: {best_val_acc*100:.2f}% -> {avg_val_acc*100:.2f}%)")
        torch.save(model.state_dict(), best_model_file)
        best_val_acc = avg_val_acc
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"  Sin mejora. {epochs_no_improve}/{EARLY_STOP_PATIENCE} épocas para Early Stop.")

    if epochs_no_improve >= EARLY_STOP_PATIENCE:
        print(f"\n--- ¡Early Stopping! ---")
        print(f"No hubo mejora en las últimas {EARLY_STOP_PATIENCE} épocas.")
        print(f"Parando entrenamiento.")
        break # Salir del bucle de Fase 2

print("\n--- Entrenamiento Completado ---")

# --- 7. Guardar Historial y Evaluar ---

# Guardar el historial completo en CSV
try:
    full_history_df.to_csv(summary_csv_file, index=False)
    print(f"Resumen final del historial COMPLETO guardado en: {summary_csv_file}")
except Exception as e:
    print(f"No se pudo guardar el resumen del historial: {e}")

# Cargar el mejor modelo para la evaluación final
try:
    print(f"\nCargando el mejor modelo guardado ({best_model_file}) para la evaluación...")
    model.load_state_dict(torch.load(best_model_file))
except Exception as e:
    print(f"No se encontró el mejor modelo en '{best_model_file}'. Usando el último modelo en memoria. Error: {e}")

model.eval() 
running_loss = 0.0
running_corrects = 0

print("Evaluando el modelo en el conjunto de prueba...")
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

test_loss = running_loss / dataset_sizes['test']
test_acc = running_corrects.double() / dataset_sizes['test']

print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}')
print(f"Precisión final en el conjunto de prueba: {test_acc * 100:.2f}%")

--- Verificando la disponibilidad de la GPU (PyTorch) ---
✅ GPU detectada: NVIDIA GeForce RTX 3060 Ti
Usando dispositivo: cuda
--------------------------------------------------
Cargando datos desde la carpeta 'C:/Users/ivanp/Desktop/organized_data/'...
Clases encontradas: ['anciano', 'joven', 'medio']
Imágenes de entrenamiento: 7095, Validación: 1521
Carga de datos finalizada.

Logs de cada época se guardarán en: training_log.csv
El mejor modelo se guardará en: best_age_model.pth

--- Iniciando Fase 1: Entrenamiento de la Capa Final (15 épocas) ---


Epoch 1/15 | Train Loss: 0.6384, Train Acc: 73.15% | Val Loss: 0.5414, Val Acc: 77.38%
  ¡Mejora! Guardando modelo... (Val Acc: 0.00% -> 77.38%)


Epoch 2/15 | Train Loss: 0.5392, Train Acc: 77.15% | Val Loss: 0.5950, Val Acc: 75.28%


Epoch 3/15 | Train Loss: 0.5201, Train Acc: 78.60% | Val Loss: 0.4547, Val Acc: 80.41%
  ¡Mejora! Guardando modelo... (Val Acc: 77.38% -> 80.41%)


Epoch 4/15 | Train Loss: 0.5028, Train Acc: 78.76% | Val Loss: 0.4582, Val Acc: 80.54%
  ¡Mejora! Guardando modelo... (Val Acc: 80.41% -> 80.54%)


...

Epoch 32/65 | Train Loss: 0.0134, Train Acc: 99.70% | Val Loss: 0.5964, Val Acc: 85.47%
  Sin mejora. 11/15 épocas para Early Stop.


Epoch 33/65 | Train Loss: 0.0106, Train Acc: 99.77% | Val Loss: 0.5811, Val Acc: 85.27%
  Sin mejora. 12/15 épocas para Early Stop.


Epoch 34/65 | Train Loss: 0.0121, Train Acc: 99.76% | Val Loss: 0.6043, Val Acc: 85.34%
  Sin mejora. 13/15 épocas para Early Stop.


Epoch 35/65 | Train Loss: 0.0124, Train Acc: 99.68% | Val Loss: 0.5828, Val Acc: 85.34%
  Sin mejora. 14/15 épocas para Early Stop.


Epoch 36/65 | Train Loss: 0.0125, Train Acc: 99.69% | Val Loss: 0.5929, Val Acc: 85.21%
  Sin mejora. 15/15 épocas para Early Stop.

--- ¡Early Stopping! ---
No hubo mejora en las últimas 15 épocas.
Parando entrenamiento.

--- Entrenamiento Completado ---
Resumen final del historial COMPLETO guardado en: training_history_summary.csv

Cargando el mejor modelo guardado (best_age_model.pth) para la evaluación...
Evaluando el modelo en el conjunto de prueba...
Test Loss: 0.4258 Acc: 0.8402
Precisión final en el conjunto de prueba: 84.02%


Obtener gráficos a partir de los resultados del entrenamiento

In [ ]:
def plot_training_history(csv_path, head_epochs, save_dir="."):
    """
    Lee el historial de entrenamiento desde un archivo CSV y genera gráficos
    de precisión y pérdida, guardándolos como archivos PNG.
    
    Args:
        csv_path (str): Ruta al archivo 'training_history_summary.csv'.
        head_epochs (int): El número de épocas de la Fase 1 (para dibujar la línea).
        save_dir (str): Directorio donde se guardarán los gráficos.
    """
    print(f"Generando gráficos desde {csv_path}...")
    
    try:
        # 1. Leer los datos del CSV
        history_df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de historial en {csv_path}")
        return
    except Exception as e:
        print(f"Error al leer el CSV: {e}")
        return

    # --- 2. Gráfico de Precisión (Accuracy) ---
    plt.figure(figsize=(12, 6))
    plt.plot(history_df['epoch'], history_df['train_acc'], label='Precisión (Entrenamiento)')
    plt.plot(history_df['epoch'], history_df['val_acc'], label='Precisión (Validación)', linestyle='--')
    
    # Dibujar línea vertical para separar las fases
    # La Fase 1 va de la época 0 a la (head_epochs - 1).
    plt.axvline(x=head_epochs - 0.5, color='grey', linestyle=':', label=f'Fin Fase 1 (Época {head_epochs-1})')
    
    plt.title('Historial de Precisión (Accuracy) del Entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.ylim(0, 1.05) # Asegurar que el eje Y va de 0 a 1 (o 1.05 para margen)
    
    # Guardar el gráfico
    acc_plot_path = os.path.join(save_dir, './train_results/training_accuracy_plot.png')
    plt.savefig(acc_plot_path)
    print(f"Gráfico de Precisión guardado en: {acc_plot_path}")
    plt.close() # Cerrar la figura para liberar memoria

    # --- 3. Gráfico de Pérdida (Loss) ---
    plt.figure(figsize=(12, 6))
    plt.plot(history_df['epoch'], history_df['train_loss'], label='Pérdida (Entrenamiento)')
    plt.plot(history_df['epoch'], history_df['val_loss'], label='Pérdida (Validación)', linestyle='--')
    
    # Dibujar línea vertical
    plt.axvline(x=head_epochs - 0.5, color='grey', linestyle=':', label=f'Fin Fase 1 (Época {head_epochs-1})')
    
    plt.title('Historial de Pérdida (Loss) del Entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.6)
    
    # Guardar el gráfico
    loss_plot_path = os.path.join(save_dir, './train_results/training_loss_plot.png')
    plt.savefig(loss_plot_path)
    print(f"Gráfico de Pérdida guardado en: {loss_plot_path}")
    plt.close()

# --- Punto de entrada principal ---
if __name__ == "__main__":
    
    # --- Configuración ---
    # Asegúrate de que estos valores coincidan con tu script de entrenamiento
    CSV_FILE_TO_LOAD = './train_results/training_history_summary.csv'
    HEAD_EPOCHS = 15 
    # --- Fin de Configuración ---
    
    plot_training_history(CSV_FILE_TO_LOAD, HEAD_EPOCHS)
    print("\nProceso de generación de gráficos finalizado.")

Generando gráficos desde ./train_results/training_history_summary.csv...
Gráfico de Precisión guardado en: .\./train_results/training_accuracy_plot.png
Gráfico de Pérdida guardado en: .\./train_results/training_loss_plot.png

Proceso de generación de gráficos finalizado.


Tras lo anterior se procede a poner un filtro dependiendo de la edad de la persona detectada a través de la WebCam

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
import mediapipe as mp # !NUEVO! Importamos MediaPipe
from PIL import Image
import os

# --- 0. Definición del Modelo (NECESARIO en PyTorch) ---
def get_model(num_classes=3):
    model = models.resnet50(weights=None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, num_classes)
    )
    return model

# --- 1. Configuración Inicial ---
IMG_SIZE = 224
MODEL_PATH = './train_results/best_age_model.pth'
LABELS = {0: "JOVEN", 1: "MEDIO", 2: "ANCIANO"}
BOX_COLOR = (0, 255, 0) 

# Rutas de filtros
FILTER_PATHS = {
    0: './filters/bebe.png',
    1: './filters/medio.png',  # Tu archivo de "bigote"
    2: './filters/anciano.png'
}

# --- 1.5. Configuración de PyTorch ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

preprocess_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- 2. Funciones Auxiliares (sin cambios) ---
def load_filter_image(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar el filtro: {path}")
    if img.shape[2] == 3:
        b_channel, g_channel, r_channel = cv2.split(img)
        alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 255
        img = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))
    return img

def overlay_image_alpha(img_base, img_overlay, x, y):
    x, y = int(x), int(y)
    h, w = img_overlay.shape[0], img_overlay.shape[1]
    y1, y2 = max(0, y), min(img_base.shape[0], y + h)
    x1, x2 = max(0, x), min(img_base.shape[1], x + w)
    y1_overlay, y2_overlay = max(0, -y), min(h, img_base.shape[0] - y)
    x1_overlay, x2_overlay = max(0, -x), min(w, img_base.shape[1] - x)
    if y1 >= y2 or x1 >= x2 or y1_overlay >= y2_overlay or x1_overlay >= x2_overlay:
        return img_base
    overlay_cropped = img_overlay[y1_overlay:y2_overlay, x1_overlay:x2_overlay]
    img_rgb = overlay_cropped[..., :3]
    alpha = overlay_cropped[..., 3:] / 255.0
    alpha_inv = 1.0 - alpha
    img_base_cropped = img_base[y1:y2, x1:x2]
    if img_base_cropped.shape != img_rgb.shape:
        return img_base
    img_base[y1:y2, x1:x2] = (img_base_cropped * alpha_inv + img_rgb * alpha).astype(np.uint8)
    return img_base

# --- 3. Cargar Modelos y Filtros ---
try:
    model_age = get_model(num_classes=len(LABELS))
    model_age.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model_age = model_age.to(device)
    model_age.eval()
    print(f"Modelo {MODEL_PATH} cargado exitosamente en {device}.")
except Exception as e:
    print(f"Error cargando el modelo PyTorch: {e}")
    exit()

loaded_filters = {}
print("Cargando imágenes de filtros...")
for age_id, path in FILTER_PATHS.items():
    try:
        loaded_filters[age_id] = load_filter_image(path)
        print(f"  Filtro '{path}' cargado para la edad {LABELS[age_id]}.")
    except FileNotFoundError as e:
        print(f"¡ADVERTENCIA! {e}. Este filtro no estará disponible.")
        loaded_filters[age_id] = None
    except Exception as e:
        print(f"Error desconocido cargando filtro '{path}': {e}. No disponible.")
        loaded_filters[age_id] = None

# --- !NUEVO! Inicializar MediaPipe ---
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

print("Iniciando cámara... Presiona 'q' para salir.")
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara web.")
    exit()

# --- 4. Bucle de Captura de Video ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Obtener alto y ancho del frame
    frame_h, frame_w, _ = frame.shape

    # Convertir a RGB para MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    try:
        # --- !MODIFICADO! Detección con MediaPipe ---
        results = face_detection.process(rgb_frame)

        if results.detections:
            for detection in results.detections:
                
                # --- !MODIFICADO! Extracción de Bounding Box ---
                # MediaPipe da coordenadas normalizadas [0, 1]
                bboxC = detection.location_data.relative_bounding_box
                x = int(bboxC.xmin * frame_w)
                y = int(bboxC.ymin * frame_h)
                w = int(bboxC.width * frame_w)
                h = int(bboxC.height * frame_h)
                x, y, w, h = max(0, x), max(0, y), max(0, w), max(0, h) # Asegurar que no sean negativos

                # --- !MODIFICADO! Extracción de Landmarks ---
                # Extraer los 6 puntos clave de MediaPipe
                keypoints = detection.location_data.relative_keypoints
                eye_left = (int(keypoints[0].x * frame_w), int(keypoints[0].y * frame_h))
                eye_right = (int(keypoints[1].x * frame_w), int(keypoints[1].y * frame_h))
                nose = (int(keypoints[2].x * frame_w), int(keypoints[2].y * frame_h))
                mouth_center = (int(keypoints[3].x * frame_w), int(keypoints[3].y * frame_h))


                try:
                    # --- 6. Procesar cada Cara (Sin cambios) ---
                    face_roi_rgb = rgb_frame[y:y+h, x:x+w]
                    
                    if face_roi_rgb.size == 0 or w <= 0 or h <= 0:
                        continue

                    pil_image = Image.fromarray(face_roi_rgb)
                    input_tensor = preprocess_transform(pil_image)
                    input_batch = input_tensor.unsqueeze(0).to(device)

                    with torch.no_grad():
                        outputs = model_age(input_batch)
                        probabilities = torch.softmax(outputs, dim=1)
                        confidence, class_id_tensor = torch.max(probabilities, 1)

                    class_id = class_id_tensor.item()
                    confidence_val = confidence.item() * 100
                    label_text = LABELS.get(class_id, "Desconocido")
                    text = f"{label_text} ({confidence_val:.1f}%)"

                    cv2.rectangle(frame, (x, y), (x+w, y+h), BOX_COLOR, 2)
                    cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, BOX_COLOR, 2)

                    # --- 8. APLICAR FILTRO VISUAL (Usando landmarks de MediaPipe) ---
                    selected_filter = loaded_filters.get(class_id)
                    if selected_filter is not None:
                        
                        if class_id == 1: # --- LÓGICA DEL BIGOTE (CLASE 1: "MEDIO") ---
                            try:
                                w_mustache = int(w * 0.6)
                                h_filter_orig, w_filter_orig = selected_filter.shape[:2]
                                if w_filter_orig == 0: continue
                                h_mustache = max(1, int(w_mustache * (h_filter_orig / w_filter_orig)))
                                
                                if w_mustache > 0 and h_mustache > 0:
                                    filter_resized = cv2.resize(selected_filter, (w_mustache, h_mustache))
                                    
                                    # Posicionar entre la nariz y la boca
                                    nose_y = nose[1] # Y del landmark de la nariz
                                    mouth_center_y = mouth_center[1] # Y del landmark de la boca
                                    center_y_between = int((nose_y + mouth_center_y) / 2)
                                    center_x_nose = nose[0] # X del landmark de la nariz
                                    
                                    x_pos = int(center_x_nose - (w_mustache / 2))
                                    y_pos = int(center_y_between - (h_mustache / 2))

                                    frame = overlay_image_alpha(frame, filter_resized, x_pos, y_pos)
                            
                            except Exception as e:
                                print(f"Error al aplicar filtro de bigote: {e}")

                        else: # --- LÓGICA DE BEBÉ (0) Y ANCIANO (2) -> EN LA FRENTE ---
                            try:
                                center_eyes_x = int((eye_left[0] + eye_right[0]) / 2)
                                center_eyes_y = int((eye_left[1] + eye_right[1]) / 2)

                                filter_width = max(1, int(w * 0.7)) 
                                h_filter_orig, w_filter_orig = selected_filter.shape[:2]
                                if w_filter_orig == 0: continue
                                filter_height = max(1, int(filter_width * (h_filter_orig / w_filter_orig)))
                                
                                if filter_width > 0 and filter_height > 0:
                                    filter_resized = cv2.resize(selected_filter, (filter_width, filter_height))
                                    
                                    forehead_center_y = int(center_eyes_y - (h * 0.30))
                                    x_pos = int(center_eyes_x - (filter_width / 2))
                                    y_pos = int(forehead_center_y - (filter_height / 2))

                                    frame = overlay_image_alpha(frame, filter_resized, x_pos, y_pos)
                            
                            except Exception as e:
                                print(f"Error al aplicar filtro de frente: {e}")
                
                except Exception as e:
                    print(f"Error procesando cara: {e}")

    except Exception as e:
        print(f"Error general en el bucle principal: {e}")

    # --- 8. Mostrar el Resultado ---
    cv2.imshow('Detector de Edad con MediaPipe (PyTorch)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 9. Limpieza ---
cap.release()
cv2.destroyAllWindows()
face_detection.close() # !NUEVO! Cerrar el modelo de MediaPipe
print("Aplicación cerrada.")

Usando dispositivo: cpu


C:\Users\ivanp\AppData\Local\Temp\ipykernel_17604\514178216.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_age.load_state_dict(torch.load(MODEL_PATH, map_locatio

Modelo ./train_results/best_age_model.pth cargado exitosamente en cpu.
Cargando imágenes de filtros...
  Filtro './filters/bebe.png' cargado para la edad JOVEN.
  Filtro './filters/medio.png' cargado para la edad MEDIO.
  Filtro './filters/anciano.png' cargado para la edad ANCIANO.
Iniciando cámara... Presiona 'q' para salir.
Aplicación cerrada.


# FILTRO

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
from scipy.spatial import distance as dist

# --- Configuración de Constantes y Estado ---

# EAR: Umbral bajo el cual el ojo se considera cerrado
EYE_AR_THRESH = 0.25 
# Número de frames consecutivos
EYE_AR_CONSEC_FRAMES = 3

STAR_IMAGE_PATH = "./filters/star.png" 
STAR_SIZE_START = 20 
STAR_SIZE_END = 80 
STAR_LIFETIME = 1.0 
STAR_SPEED = 20 

# Estado
active_stars = [] 
COUNTER = 0 
BLINKS = 0 

# --- Índices de MediaPipe para los ojos ---
# MediaPipe usa una malla de 468 puntos. Estos son los índices correspondientes a los ojos.
# Orden: [p1, p2, p3, p4, p5, p6] donde p1 y p4 son las esquinas horizontales
LEFT_EYE_IDX = [362, 385, 387, 263, 373, 380]
RIGHT_EYE_IDX = [33, 160, 158, 133, 153, 144]

# --- Funciones Auxiliares ---

def eye_aspect_ratio(eye_points):
    # Calcula la distancia euclidiana vertical
    A = dist.euclidean(eye_points[1], eye_points[5])
    B = dist.euclidean(eye_points[2], eye_points[4])
    # Calcula la distancia euclidiana horizontal
    C = dist.euclidean(eye_points[0], eye_points[3])

    # Calcula el EAR
    ear = (A + B) / (2.0 * C)
    return ear

# --- Inicialización ---

# 1. Cargar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True, # Importante para mayor precisión en ojos
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 2. Cargar Imagen Estrella
try:
    star_img_orig = cv2.imread(STAR_IMAGE_PATH, cv2.IMREAD_COLOR)
    if star_img_orig is None:
        print(f"[AVISO] No se encontró '{STAR_IMAGE_PATH}'. El efecto no se mostrará.")
    else:
        print(f"[INFO] Immagine stellina caricata.")
except Exception as e:
    print(f"[ERRORE] {e}")
    star_img_orig = None

# 3. Iniciar Webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Errore: Impossibile aprire la webcam.")
    exit()

print("[INFO] Avvio del ciclo video con MediaPipe...")

# --- Ciclo Principal ---

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Espejo y dimensiones
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    
    # MediaPipe necesita RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Procesar detección
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Convertir landmarks normalizados a píxeles
            mesh_points = np.array([np.multiply([p.x, p.y], [w, h]).astype(int) for p in face_landmarks.landmark])

            # Obtener coordenadas de los ojos
            leftEye = mesh_points[LEFT_EYE_IDX]
            rightEye = mesh_points[RIGHT_EYE_IDX]

            # Calcular EAR
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # Dibujar contornos de ojos (Opcional)
            cv2.polylines(frame, [leftEye], True, (0, 255, 0), 1)
            cv2.polylines(frame, [rightEye], True, (0, 255, 0), 1)

            # --- Lógica de Parpadeo ---
            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    BLINKS += 1
                    
                    # Generar estrellas si la imagen existe
                    if star_img_orig is not None:
                        le_center = np.mean(leftEye, axis=0).astype(int)
                        re_center = np.mean(rightEye, axis=0).astype(int)

                        active_stars.append({
                            'start_time': time.time(),
                            'x': le_center[0], 'y': le_center[1],
                            'initial_pos': le_center
                        })
                        active_stars.append({
                            'start_time': time.time(),
                            'x': re_center[0], 'y': re_center[1],
                            'initial_pos': re_center
                        })
                
                COUNTER = 0

    # --- Renderizado de Animación (Independiente de la detección facial) ---
    current_time = time.time()
    stars_to_keep = []

    for star in active_stars:
        elapsed_time = current_time - star['start_time']

        if elapsed_time < STAR_LIFETIME:
            scale_factor = elapsed_time / STAR_LIFETIME
            current_size = int(STAR_SIZE_START + (STAR_SIZE_END - STAR_SIZE_START) * scale_factor)
            alpha = 1.0 - scale_factor

            # Movimiento
            dir_x = (star['x'] - star['initial_pos'][0])
            dir_y = (star['y'] - star['initial_pos'][1])
            
            # Si es el primer frame, dar una dirección aleatoria hacia arriba/afuera o fija
            # Aquí mantengo tu lógica original, que asume movimiento radial, 
            # pero al nacer en el centro, dir_x es 0. Forzamos un movimiento hacia arriba/lados
            if dir_x == 0 and dir_y == 0:
                 # Truco: Pequeño offset aleatorio para que no se queden quietas
                 dir_x = np.random.choice([-1, 1]) 
                 dir_y = -1 # Hacia arriba

            magnitude = math.sqrt(dir_x**2 + dir_y**2)
            norm_dir_x, norm_dir_y = (dir_x / magnitude, dir_y / magnitude) if magnitude > 0 else (0, -1)
            
            move_distance = STAR_SPEED * elapsed_time
            # Actualizamos posición base + movimiento
            # Nota: Tu lógica original movía la estrella basándose en la posición previa implicita
            # Aquí simplifico para que se mueva radialmente desde el ojo
            new_x = int(star['initial_pos'][0] + norm_dir_x * move_distance * 20) # *50 factor arbitrario de velocidad visual
            new_y = int(star['initial_pos'][1] + norm_dir_y * move_distance * 20)

            # Renderizado (Blending)
            if star_img_orig is not None and current_size > 0:
                resized_star = cv2.resize(star_img_orig, (current_size, current_size))
                
                x1 = new_x - current_size // 2
                y1 = new_y - current_size // 2
                x2 = x1 + current_size
                y2 = y1 + current_size

                y1_frame = max(0, y1)
                x1_frame = max(0, x1)
                y2_frame = min(frame.shape[0], y2)
                x2_frame = min(frame.shape[1], x2)

                y1_star = y1_frame - y1
                x1_star = x1_frame - x1
                y2_star = y1_star + (y2_frame - y1_frame)
                x2_star = x1_star + (x2_frame - x1_frame)

                if x2_frame > x1_frame and y2_frame > y1_frame:
                    roi_frame = frame[y1_frame:y2_frame, x1_frame:x2_frame].astype(np.float32)
                    roi_star = resized_star[y1_star:y2_star, x1_star:x2_star].astype(np.float32)
                    
                    blended = roi_frame * (1.0 - alpha) + roi_star * alpha
                    frame[y1_frame:y2_frame, x1_frame:x2_frame] = blended.astype(np.uint8)

            stars_to_keep.append(star)
    
    active_stars = stars_to_keep

    # UI Info
    cv2.putText(frame, f"Blinks: {BLINKS}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow("MediaPipe Blink Filter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[INFO] Immagine stellina caricata.
[INFO] Avvio del ciclo video con MediaPipe...
